In [1]:
ratio_ViT_B_16          = 0.55
ratio_CLIP_Interrogator = 0.45

# CLIP Interrrogator

In [2]:
!pip install torch clip

ERROR: Could not find a version that satisfies the requirement clip (from versions: none)
ERROR: No matching distribution found for clip


In [3]:
import torch
import sys
from pathlib import Path
import pandas as pd
import os
import numpy as np
from PIL import Image

wheels_path = "/kaggle/input/clip-interrogator-wheels-x"
clip_interrogator_whl_path = f"{wheels_path}/clip_interrogator-0.4.3-py3-none-any.whl"
!pip install --no-index --find-links $wheels_path $clip_interrogator_whl_path -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
tfx-bsl 1.9.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.52.0 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.8.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires typing-extensions<3.11,>=3.7, but you have typing-extensions 4.1.1 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*

In [4]:
!pip install --no-index --find-links $wheels_path $clip_interrogator_whl_path -q

In [5]:
!pip install torch clip

ERROR: Could not find a version that satisfies the requirement clip (from versions: none)
ERROR: No matching distribution found for clip


In [6]:
import torch
import sys
from pathlib import Path
import pandas as pd
import os
import numpy as np
from PIL import Image
def make_batches(l, batch_size=16):
    for i in range(0, len(l), batch_size):
        yield l[i:i + batch_size]

path = Path('../input/stable-diffusion-image-to-prompts/')
# sample_submission = pd.read_csv(path / 'sample_submission.csv', index_col='imgId_eId')
images = os.listdir(path/"images")
# image_ids = [i.split('.')[0] for i in images]

device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
!pip list | grep transformers


transformers                          4.15.0


In [8]:
class CFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    sentence_model_path = "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    blip_model_path = "/kaggle/input/clip-interrogator-models-x/model_large_caption.pth"
    ci_clip_model_name = "ViT-H-14/laion2b_s32b_b79k"
    clip_model_name = "ViT-H-14"
    clip_model_path = "/kaggle/input/clip-interrogator-models-x/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_pytorch_model.bin"
    cache_path = "/kaggle/input/clip-interrogator-models-x"
import open_clip
import inspect
import importlib

from blip.models import blip
from clip_interrogator import clip_interrogator
from PIL import Image
import torchvision.transforms as transforms
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

st_model = SentenceTransformer(CFG.sentence_model_path)

In [9]:
# replace tokenizer path to prevent downloading
# blip 모델 모듈 대체
blip_path = inspect.getfile(blip)

fin = open(blip_path, "rt")
data = fin.read()
data = data.replace(
    "BertTokenizer.from_pretrained('bert-base-uncased')", 
    "BertTokenizer.from_pretrained('/kaggle/input/clip-interrogator-models-x/bert-base-uncased')"
)
fin.close()

fin = open(blip_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(blip)

<module 'blip.models.blip' from '/opt/conda/lib/python3.7/site-packages/blip/models/blip.py'>

In [10]:
# fix clip_interrogator bug
clip_interrogator_path = inspect.getfile(clip_interrogator.Interrogator)

fin = open(clip_interrogator_path, "rt")
data = fin.read()
data = data.replace(
    'open_clip.get_tokenizer(clip_model_name)', 
    'open_clip.get_tokenizer(config.clip_model_name.split("/", 2)[0])'
)
fin.close()

fin = open(clip_interrogator_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(clip_interrogator)

<module 'clip_interrogator.clip_interrogator' from '/opt/conda/lib/python3.7/site-packages/clip_interrogator/clip_interrogator.py'>

In [11]:
# Step 1: Config 객체 생성
model_config = clip_interrogator.Config(clip_model_name=CFG.ci_clip_model_name)
model_config.cache_path = CFG.cache_path

In [12]:
configs_path = os.path.join(os.path.dirname(os.path.dirname(blip_path)), 'configs')
med_config = os.path.join(configs_path, 'med_config.json')
blip_model = blip.blip_decoder(
    pretrained=CFG.blip_model_path,
    image_size=model_config.blip_image_eval_size, 
    vit=model_config.blip_model_type, 
    med_config=med_config
)
blip_model.eval()
blip_model = blip_model.to(model_config.device)
model_config.blip_model = blip_model

load checkpoint from /kaggle/input/clip-interrogator-models-x/model_large_caption.pth


In [13]:
# CLIP 모델 생성
clip_model = open_clip.create_model(CFG.clip_model_name, precision='fp16' if model_config.device == 'cuda' else 'fp32')

# #학습된 모델 불러오기
open_clip.load_checkpoint(clip_model, CFG.clip_model_path)
clip_model.to(model_config.device).eval()
model_config.clip_model = clip_model

In [14]:
clip_preprocess = transforms.Compose([
     transforms.Resize((224, 224)),
     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
         mean=[0.5, 0.5, 0.5],
         std=[0.5, 0.5, 0.5]
        #transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
     )
 ])
model_config.clip_preprocess = clip_preprocess
# # transforms.Normalize(
# #         mean=[0.5, 0.5, 0.5],
# #         std=[0.229, 0.224, 0.225]

In [15]:
# Step 2: Interrogator 객체 생성
ci = clip_interrogator.Interrogator(model_config)

Loaded CLIP model and data in 2.38 seconds.


In [16]:
def image_to_features(image: Image) -> torch.Tensor:
        ci._prepare_clip()
        images = ci.clip_preprocess(image).unsqueeze(0).to(ci.device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = ci.clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
        return image_features
    

In [17]:
def interrogate_classic(image: Image,max_flavors: int=32) -> str:
        """Classic mode creates a prompt in a standard format first describing the image, 
        then listing the artist, trending, movement, and flavor text modifiers."""
        caption = ci.generate_caption(image)
        image_features = ci.image_to_features(image)

        medium = ci.mediums.rank(image_features, 1)[0]
        movement = ci.movements.rank(image_features, 1)[0]
        flaves = ", ".join(ci.flavors.rank(image_features, max_flavors))

        if caption.startswith(medium):
            prompt = f"{caption} , {movement}, {flaves}"
        else:
            prompt = f"{caption}, {medium} , {movement}, {flaves}"

        return clip_interrogator._truncate_to_fit(prompt, ci.tokenize)

In [18]:
# #이미지에 적용
prompts = []

images_path = "../input/stable-diffusion-image-to-prompts/images/"
for image_name in images:
    img = Image.open(images_path + image_name).convert("RGB")

    generated = interrogate_classic(img)
    
    prompts.append(generated)

100%|██████████| 50/50 [00:00<00:00, 202.69it/s]


In [19]:
df_submission = pd.read_csv(path / 'sample_submission.csv', index_col='imgId_eId')
df_submission.head()

,val
imgId_eId,
20057f34d_0,0.018848
20057f34d_1,0.030190
20057f34d_2,0.072792
20057f34d_3,-0.000673
20057f34d_4,0.016774


In [20]:
images = os.listdir(path / 'images')
imgIds = [i.split('.')[0] for i in images]

eIds = list(range(CFG.embedding_length))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, CFG.embedding_length),
        np.tile(range(CFG.embedding_length), len(imgIds))
    )
]

assert sorted(imgId_eId) == sorted(df_submission.index)

In [21]:
embeddings2 = st_model.encode(prompts).flatten()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# VIT 

In [22]:
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.preprocessing import normalize

In [23]:
class DiffusionTestDataset(Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        image = self.transform(image)
        return image

In [24]:
def predict(
    images,
    model_path,
    model_name,
    input_size,
    batch_size,
    learning_rate
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomRotation(degrees=10),

        # transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    dataset = DiffusionTestDataset(images, transform)
    dataloader = DataLoader(
        dataset=dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False
    )

    model = timm.create_model(
        model_name,
        pretrained=False,
        num_classes=384
    )
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    
    tta_preds = None
    for _ in range(2):
        preds = []
        for X in tqdm(dataloader, leave=False):
            X = X.to(device)

            with torch.no_grad():
                X_out = model(X).cpu().numpy()
                # L2 normalize -- Start
                X_out = X_out / ( np.abs(X_out).max(axis=-1, keepdims=True) + 0.0000001)  # To avoid to overflow at normalize()
                X_out = normalize( X_out )
                # L2 normalize -- End
                preds.append(X_out)
                
        if tta_preds is None:
            tta_preds = np.vstack(preds).flatten()
        else:
            tta_preds += np.vstack(preds).flatten()
    
    return tta_preds / 2

In [25]:
import itertools

# 하이퍼파라미터 조합 생성
param_grid = {
    'model_path': ['/kaggle/input/stable-diffusion-vit-baseline-train/vit_base_patch16_224.pth'],
    'model_name': ['vit_base_patch16_224'],
    'input_size': [224],
    'batch_size': [16, 32, 64, 128, 256],
    'patch_size': [8, 16, 32],
    'learning_rate': [1e-5, 1e-4, 1e-3]
}

param_list = list(itertools.product(*param_grid.values()))

# 결과 저장을 위한 딕셔너리 생성
results = {}

# 하이퍼파라미터 조합마다 predict 수행
for params in param_list:
    # 딕셔너리 형태로 변환
    params_dict = {list(param_grid.keys())[i]: params[i] for i in range(len(params))}
    print("Testing params:", params_dict)
    
    # predict 수행
    embeddings = predict(images, params_dict['model_path'], params_dict['model_name'], 
                         params_dict['input_size'], params_dict['batch_size'], params_dict['learning_rate'])
    
    # 결과 저장
    results[str(params_dict)] = embeddings.mean()
    
# 결과 출력
print("Results:", results)

# 최적의 조합 찾기
best_params = max(results, key=results.get)
print("Best params:", best_params)

In [26]:
class CFG:
    model_path = '/kaggle/input/stable-diffusion-vit-baseline-train/vit_base_patch16_224.pth'
    model_name = 'vit_base_patch16_224'
    input_size = 224
    batch_size = 256
    patch_size = 8
    learning_rate = 0.001

In [27]:
images = list(Path('/kaggle/input/stable-diffusion-image-to-prompts/images').glob('*.png'))
imgIds = [i.stem for i in images]
EMBEDDING_LENGTH = 384
imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, EMBEDDING_LENGTH),
        np.tile(range(EMBEDDING_LENGTH), len(imgIds)))]

embeddings3 = predict(images, CFG.model_path, CFG.model_name, CFG.input_size, CFG.batch_size, CFG.learning_rate)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# ENSEMBLE THE OUTPUTS

In [28]:
embeddings = embeddings2*ratio_CLIP_Interrogator + ratio_ViT_B_16 * embeddings3

In [29]:
submission = pd.DataFrame(
    index=imgId_eId,
    data=embeddings,
    columns=['val']
).rename_axis('imgId_eId')

In [30]:
submission.head()

,val
imgId_eId,
f27825b2c_0,-0.030706
f27825b2c_1,0.045031
f27825b2c_2,-0.031132
f27825b2c_3,-0.045387
f27825b2c_4,-0.060089


In [31]:
submission.to_csv('submission.csv')